In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from evaluate import calibrate_results, generate_everything_all_epochs

In [ ]:
############################################
# select best model/epoch based on a specific group performance
#
# for dp
############################################

def select_model(exp_folder_name, epochs, group_name):
    """
    select model based on bal_acc and pr c1
    
    exp_folder_name: the output directory for the experiment
    there should be one file containing the model prediction 
    on the validation set and one file containing prediction 
    on the test set for each epoch/model
    
    cancer: type of cancer. bcs and lcs have different minority class
    """
    all_epochs = []
    
    for e in range(1, epochs+1):
        # select based on validation set, whole group threshold
        df = pd.read_csv(exp_folder_name + '/epo ' + str(e) + '/valid_subgroup_threshold_summary.csv')
        # base on subgroup
        all_epochs.append(df.loc[df['Unnamed: 0'] == group_name].values[0])
        
    cols = ['Group', 'Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    df_all_epochs = pd.DataFrame(data=all_epochs, index=range(1, epochs+1), columns=cols)
    df_all_epochs.to_csv(exp_folder_name + '/all_epoch_metrics.csv')
    
    df_top_3 = df_all_epochs.nlargest(3, 'Bal_Acc')
    df_best = df_top_3.nlargest(1, 'PR_C1')
    best_index = df_best.index.tolist()[0]
    output = open(exp_folder_name + '/best_epoch_number_' + group_name + '.txt', 'w')
    output.write(str(best_index))
    output.close()

In [ ]:
def select_model_whole_group(exp_folder_name, epochs, group_name):
    """
    select model based on bal_acc and pr c1
    
    exp_folder_name: the output directory for the experiment
    there should be one file containing the model prediction 
    on the validation set and one file containing prediction 
    on the test set for each epoch/model
    
    cancer: type of cancer. bcs and lcs have different minority class
    """
    all_epochs = []
    
    for e in range(1, epochs+1):
        # select based on validation set, whole group threshold
        df = pd.read_csv(exp_folder_name + '/epo ' + str(e) + '/valid_whole_group_threshold_summary.csv')
        # base on subgroup
        all_epochs.append(df.loc[df['Unnamed: 0'] == group_name].values[0])
        
    cols = ['Group', 'Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    df_all_epochs = pd.DataFrame(data=all_epochs, index=range(1, epochs+1), columns=cols)
    df_all_epochs.to_csv(exp_folder_name + '/all_epoch_metrics_whole_group_thre.csv')
    
    df_top_3 = df_all_epochs.nlargest(3, 'Bal_Acc')
    df_best = df_top_3.nlargest(1, 'PR_C1')
    best_index = df_best.index.tolist()[0]
    output = open(exp_folder_name + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'w')
    output.write(str(best_index))
    output.close()

In [ ]:
def eval_all_units(folder_name, cancer, group_name):
    units = ['2u', '3u', '4u', '5u', '6u', '7u', '8u', '9u', '10u', '12u', '14u', '16u', '18u', '20u']
    
    for u in units:
        folder = folder_name + '/' + folder_name.split('/')[-1][:-2] + '-' + u
        
        calibrate_results(folder, 25)
        generate_everything_all_epochs(folder, 25, cancer)
        select_model(folder, 25, group_name)

In [ ]:
def eval_all_units_whole_group(folder_name, cancer, group_name):
    units = ['2u', '3u', '4u', '5u', '6u', '7u', '8u', '9u', '10u', '12u', '14u', '16u', '18u', '20u']
    
    for u in units:
        folder = folder_name + '/' + folder_name.split('/')[-1][:-2] + '-' + u
        
        calibrate_results(folder, 25)
        generate_everything_all_epochs(folder, 25, cancer)
        select_model_whole_group(folder, 25, group_name)

In [ ]:
def select_dp_unit_subgroup_threshold(dp_folder_name, original_folder_name, group_name):
    """
    select the best unit (from 1 to 20) of dp
    original model is the model containing 1 unit of minority samples
    dp_folder_name: the folder containing dp 2-20 units
    original_folder_name: the folder containing the original model (1 unit)
    group_name: the subgroup name dp is applied to
    """
    all_units = []
    all_units_test = []
    
    # read the best epoch of each unit model
    # and extract perfomance metrics
    df_orig = pd.read_csv(original_folder_name + '/valid_subgroup_threshold_summary_mean.csv')
    df_orig_test = pd.read_csv(original_folder_name + '/test_subgroup_threshold_summary_mean.csv')
    all_units.append(df_orig.loc[df_orig['Unnamed: 0'] == group_name].values[0]) # subgroup performance
    all_units_test.append(df_orig_test.loc[df_orig_test['Unnamed: 0'] == group_name].values[0])
    
    units = ['2u', '3u', '4u', '5u', '6u', '7u', '8u', '9u', '10u', '12u', '14u', '16u', '18u', '20u']
    
    for u in units:
        unit_prefix = dp_folder_name + '/' + dp_folder_name.split('/')[-1][:-2] + '-' + u
        file = open(unit_prefix + '/best_epoch_number_' + group_name + '.txt', 'r')
        best_epo = file.read()
        df = pd.read_csv(unit_prefix + '/epo ' + best_epo + '/valid_subgroup_threshold_summary.csv')
        df_test = pd.read_csv(unit_prefix + '/epo ' + best_epo + '/test_subgroup_threshold_summary.csv')
        all_units.append(df.loc[df['Unnamed: 0'] == group_name].values[0]) # subgroup performance
        all_units_test.append(df_test.loc[df_test['Unnamed: 0'] == group_name].values[0])
    
    # picking top 3 bal_acc and then the best pr c1
    cols = ['Group', 'Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    rows = ['1 unit', '2 units', '3 units', '4 units', '5 units', '6 units', '7 units', '8 units', '9 units', \
            '10 units', '12 units', '14 units', '16 units', '18 units', '20 units']
    df_all_units = pd.DataFrame(data=all_units, index=rows, columns=cols)
    df_all_units.to_csv(dp_folder_name + '/all_units_subgroup_threshold_valid.csv')
    df_all_units_test = pd.DataFrame(data=all_units_test, index=rows, columns=cols)
    df_all_units_test.to_csv(dp_folder_name + '/all_units_subgroup_threshold_test.csv')
    
    df_best = df_all_units.nlargest(3, 'Bal_Acc').nlargest(1, 'PR_C1')
    
    # save the unit number
    best_index = df_best.index.tolist()[0]
    output = open(dp_folder_name + '/best_dp_unit_number_subgroup_threshold.txt', 'w')
    output.write(str(best_index))
    output.close()

In [ ]:
def select_dp_unit_whole_group_threshold(dp_folder_name, original_folder_name, group_name):
    """
    select the best unit (from 1 to 20) of dp
    original model is the model containing 1 unit of minority samples
    dp_folder_name: the folder containing dp 2-20 units
    original_folder_name: the folder containing the original model (1 unit)
    group_name: the subgroup name dp is applied to
    """
    all_units = []
    all_units_test = []
    
    # read the best epoch of each unit model
    # and extract perfomance metrics
    
    # whole group threshold, subgroup performance
    df_orig = pd.read_csv(original_folder_name + '/valid_whole_group_threshold_summary_mean.csv')
    df_orig_test = pd.read_csv(original_folder_name + '/test_whole_group_threshold_summary_mean.csv')
    all_units.append(df_orig.loc[df_orig['Unnamed: 0'] == group_name].values[0]) # subgroup performance
    all_units_test.append(df_orig_test.loc[df_orig_test['Unnamed: 0'] == group_name].values[0])
    
    units = ['2u', '3u', '4u', '5u', '6u', '7u', '8u', '9u', '10u', '12u', '14u', '16u', '18u', '20u']
    group_name2 = dp_folder_name.split(' ')[-1] # last word in folder name is the group name
    
    for u in units:
        unit_prefix = dp_folder_name + '/' + dp_folder_name.split('/')[-1][:-2] + '-' + u
        file = open(unit_prefix + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r')
        best_epo = file.read()
        df = pd.read_csv(unit_prefix + '/epo ' + best_epo + '/valid_whole_group_threshold_summary.csv')
        df_test = pd.read_csv(unit_prefix + '/epo ' + best_epo + '/test_whole_group_threshold_summary.csv')
        all_units.append(df.loc[df['Unnamed: 0'] == group_name].values[0]) # subgroup performance
        all_units_test.append(df_test.loc[df_test['Unnamed: 0'] == group_name].values[0])
    
    # picking top 3 bal_acc and then the best pr c1
    cols = ['Group', 'Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    rows = ['1 unit', '2 units', '3 units', '4 units', '5 units', '6 units', '7 units', '8 units', '9 units', \
            '10 units', '12 units', '14 units', '16 units', '18 units', '20 units']
    df_all_units = pd.DataFrame(data=all_units, index=rows, columns=cols)
    df_all_units.to_csv(dp_folder_name + '/all_units_whole_group_threshold_valid.csv')
    df_all_units_test = pd.DataFrame(data=all_units_test, index=rows, columns=cols)
    df_all_units_test.to_csv(dp_folder_name + '/all_units_whole_group_threshold_test.csv')
    
    df_best = df_all_units.nlargest(3, 'Bal_Acc').nlargest(1, 'PR_C1')
    
    # save the unit number
    best_index = df_best.index.tolist()[0]
    output = open(dp_folder_name + '/best_dp_unit_number_whole_group_threshold.txt', 'w')
    output.write(str(best_index))
    output.close()

In [ ]:
def average_5_runs_dp_subgroup_threshold(dp_folder_name, original_folder_name, group_name):
    
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        folder_each_run = dp_folder_name + '/' + dp_folder_name.split('/')[-1] + '-' + str(i)
        
        # open best unit
        unit_file = open(folder_each_run + '/best_dp_unit_number_subgroup_threshold.txt', 'r')
        best_unit = unit_file.read().split(' ')[0] # number only
        
        print(best_unit)
        
        # open best epoch
        if best_unit == '1':
            df = pd.read_csv(original_folder_name + '/valid_subgroup_threshold_summary_mean.csv')
            df_test = pd.read_csv(original_folder_name + '/test_subgroup_threshold_summary_mean.csv')
            
        else:
            best_unit_folder = folder_each_run + '/' + dp_folder_name.split('/')[-1] + '-' + best_unit + 'u'
            epoch_file = open(best_unit_folder + '/best_epoch_number_' + group_name + '.txt', 'r')
            best_epo = epoch_file.read()
        
            # read summary files
            df = pd.read_csv(best_unit_folder + '/epo ' + best_epo + '/valid_subgroup_threshold_summary.csv')
            df_test = pd.read_csv(best_unit_folder + '/epo ' + best_epo + '/test_subgroup_threshold_summary.csv')

        df = df.drop(columns=['Unnamed: 0'])
        df_test = df_test.drop(columns=['Unnamed: 0'])
        
        valid_arr.append(df.values.flatten())
        test_arr.append(df_test.values.flatten())
        
    valid_mean = np.mean(valid_arr, axis=0)
    valid_std = np.std(valid_arr, axis=0)

    valid_mean = valid_mean.reshape(15, 12)
    valid_std = valid_std.reshape(15, 12)
    
    test_mean = np.mean(test_arr, axis=0)
    test_std = np.std(test_arr, axis=0)

    test_mean = test_mean.reshape(15, 12)
    test_std = test_std.reshape(15, 12)

    cols = ['Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    rows = ['Whole', 'Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Black',\
             'Ethnicity_Hispanic', 'Ethnicity_Asian',\
             'Age<30', '30<=Age<40', '40<=Age<50',\
             '50<=Age<60', '60<=Age<70',\
             '70<=Age<80', '80<=Age<90', 'Age>=90']

    df_valid_mean = pd.DataFrame(data=valid_mean, index=rows, columns=cols)
    df_valid_std = pd.DataFrame(data=valid_std, index=rows, columns=cols)
    df_test_mean = pd.DataFrame(data=test_mean, index=rows, columns=cols)
    df_test_std = pd.DataFrame(data=test_std, index=rows, columns=cols)

    df_valid_mean.to_csv(dp_folder_name + '/valid_subgroup_threshold_summary_mean.csv')
    df_valid_std.to_csv(dp_folder_name + '/valid_subgroup_threshold_summary_std.csv')
    df_test_mean.to_csv(dp_folder_name + '/test_subgroup_threshold_summary_mean.csv')
    df_test_std.to_csv(dp_folder_name + '/test_subgroup_threshold_summary_std.csv')

In [ ]:
def average_5_runs_dp_whole_group_threshold(dp_folder_name, original_folder_name, group_name):
    
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        folder_each_run = dp_folder_name + '/' + dp_folder_name.split('/')[-1] + '-' + str(i)
        
        # open best unit
        unit_file = open(folder_each_run + '/best_dp_unit_number_whole_group_threshold.txt', 'r')
        best_unit = unit_file.read().split(' ')[0] # number only
        
        print(best_unit)
        
        # open best epoch
        if best_unit == '1':
            df = pd.read_csv(original_folder_name + '/valid_whole_group_threshold_summary_mean.csv')
            df_test = pd.read_csv(original_folder_name + '/test_whole_group_threshold_summary_mean.csv')
            
        else:
            best_unit_folder = folder_each_run + '/' + dp_folder_name.split('/')[-1] + '-' + best_unit + 'u'
            epoch_file = open(best_unit_folder + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r')
            best_epo = epoch_file.read()
        
            # read summary files
            df = pd.read_csv(best_unit_folder + '/epo ' + best_epo + '/valid_whole_group_threshold_summary.csv')
            df_test = pd.read_csv(best_unit_folder + '/epo ' + best_epo + '/test_whole_group_threshold_summary.csv')

        df = df.drop(columns=['Unnamed: 0'])
        df_test = df_test.drop(columns=['Unnamed: 0'])
        
        valid_arr.append(df.values.flatten())
        test_arr.append(df_test.values.flatten())
        
    valid_mean = np.mean(valid_arr, axis=0)
    valid_std = np.std(valid_arr, axis=0)

    valid_mean = valid_mean.reshape(15, 12)
    valid_std = valid_std.reshape(15, 12)
    
    test_mean = np.mean(test_arr, axis=0)
    test_std = np.std(test_arr, axis=0)

    test_mean = test_mean.reshape(15, 12)
    test_std = test_std.reshape(15, 12)

    cols = ['Rec_C1', 'Prec_C1', 'PR_C1', 'F1_C1', 'Rec_C0', 'Prec_C0', 'PR_C0', 'F1_C0', 'Acc', 'Bal_Acc', 'ROC', 'threshold']
    rows = ['Whole', 'Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Black',\
             'Ethnicity_Hispanic', 'Ethnicity_Asian',\
             'Age<30', '30<=Age<40', '40<=Age<50',\
             '50<=Age<60', '60<=Age<70',\
             '70<=Age<80', '80<=Age<90', 'Age>=90']

    df_valid_mean = pd.DataFrame(data=valid_mean, index=rows, columns=cols)
    df_valid_std = pd.DataFrame(data=valid_std, index=rows, columns=cols)
    df_test_mean = pd.DataFrame(data=test_mean, index=rows, columns=cols)
    df_test_std = pd.DataFrame(data=test_std, index=rows, columns=cols)

    df_valid_mean.to_csv(dp_folder_name + '/valid_whole_group_threshold_summary_mean.csv')
    df_valid_std.to_csv(dp_folder_name + '/valid_whole_group_threshold_summary_std.csv')
    df_test_mean.to_csv(dp_folder_name + '/test_whole_group_threshold_summary_mean.csv')
    df_test_std.to_csv(dp_folder_name + '/test_whole_group_threshold_summary_std.csv')

In [ ]:
def output_5_runs(dp_folder_name, original_folder_name, group_name):
    """
    put results of each individual runs into 1 file
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        folder_each_run = dp_folder_name + '/' + dp_folder_name.split('/')[-1] + '-' + str(i)
        
        # open best unit
        unit_file = open(folder_each_run + '/best_dp_unit_number_whole_group_threshold.txt', 'r')
        best_unit = unit_file.read().split(' ')[0] # number only
        unit_file.close()
        
         # open best epoch
        if best_unit == '1':
            with open(original_folder_name + '/valid_whole_group_threshold_summary_mean.csv', 'r') as v:
                valid_arr.append(v.read())
            with open(original_folder_name + '/test_whole_group_threshold_summary_mean.csv', 'r') as t:
                test_arr.append(t.read())
                
        else:
            best_unit_folder = folder_each_run + '/' + dp_folder_name.split('/')[-1] + '-' + best_unit + 'u'
            epoch_file = open(best_unit_folder + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r')
            best_epo = epoch_file.read()
        
            # read summary files
            with open(best_unit_folder + '/epo ' + best_epo + '/valid_whole_group_threshold_summary.csv') as v:
                valid_arr.append(v.read())
            with open(best_unit_folder + '/epo ' + best_epo + '/test_whole_group_threshold_summary.csv') as t:
                test_arr.append(t.read())

        
    with open(dp_folder_name + '/valid_5_run_results.csv', 'w') as vout:
        vout.write('\n\n'.join(valid_arr))
        
    with open(dp_folder_name + '/test_5_run_results.csv', 'w') as tout:
        tout.write('\n\n'.join(test_arr))

In [ ]:
########
# whole group threshold, subgroup performance

In [ ]:
output_5_runs('../experiments/bcs-dp-asian', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
output_5_runs('../experiments/bcs-dp-hispanic', '../experiments/bcs-original', 'Ethnicity_Hispanic')

In [ ]:
output_5_runs('../experiments/bcs-dp-age30', '../experiments/bcs-original', 'Age<30')
output_5_runs('../experiments/bcs-dp-4050', '../experiments/bcs-original', '40<=Age<50')

In [ ]:
output_5_runs('../experiments/bcs-dpreweight-4050', '../experiments/bcs-original', '40<=Age<50')
output_5_runs('../experiments/bcs-dpreweight-asian', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
output_5_runs('../experiments/bcs-dp-age90', '../experiments/bcs-original', 'Age>=90')
output_5_runs('../experiments/bcs-dp-black', '../experiments/bcs-original', 'Ethnicity_Black')

In [ ]:
output_5_runs('../experiments/lcs-dp-asian', '../experiments/lcs-original', 'Ethnicity_Asian')
output_5_runs('../experiments/lcs-dp-hispanic', '../experiments/lcs-original', 'Ethnicity_Hispanic')
output_5_runs('../experiments/lcs-dp-age90', '../experiments/lcs-original', 'Age>=90')
output_5_runs('../experiments/lcs-dp-8090', '../experiments/lcs-original', '80<=Age<90')
output_5_runs('../experiments/lcs-dp-3040', '../experiments/lcs-original', '30<=Age<40')
output_5_runs('../experiments/lcs-dp-black', '../experiments/lcs-original', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-asian/bcs-dp-asian-' + str(i), 'breast', 'Ethnicity_Asian')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-asian/bcs-dp-asian-' + str(i), '../experiments/bcs-original', 'Ethnicity_Asian')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-asian', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-black/bcs-dp-black-' + str(i), 'breast', 'Ethnicity_Black')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-black/bcs-dp-black-' + str(i), '../experiments/bcs-original', 'Ethnicity_Black')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-black', '../experiments/bcs-original', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-' + str(i), 'breast', 'Ethnicity_Hispanic')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-' + str(i), '../experiments/bcs-original', 'Ethnicity_Hispanic')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-hispanic', '../experiments/bcs-original', 'Ethnicity_Hispanic')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-age30/bcs-dp-age30-' + str(i), 'breast', 'Age<30')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-age30/bcs-dp-age30-' + str(i), '../experiments/bcs-original', 'Age<30')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-age30', '../experiments/bcs-original', 'Age<30')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-4050/bcs-dp-4050-' + str(i), 'breast', '40<=Age<50')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-4050/bcs-dp-4050-' + str(i), '../experiments/bcs-original', '40<=Age<50')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-4050', '../experiments/bcs-original', '40<=Age<50')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/bcs-dp-age90/bcs-dp-age90-' + str(i), 'breast', 'Age>=90')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dp-age90/bcs-dp-age90-' + str(i), '../experiments/bcs-original', 'Age>=90')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dp-age90', '../experiments/bcs-original', 'Age>=90')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-black/lcs-dp-black-' + str(i), 'lung', 'Ethnicity_Black')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-black/lcs-dp-black-' + str(i), '../experiments/lcs-original', 'Ethnicity_Black')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-black', '../experiments/lcs-original', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-asian/lcs-dp-asian-' + str(i), 'lung', 'Ethnicity_Asian')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-asian/lcs-dp-asian-' + str(i), '../experiments/lcs-original', 'Ethnicity_Asian')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-asian', '../experiments/lcs-original', 'Ethnicity_Asian')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-hispanic/lcs-dp-hispanic-' + str(i), 'lung', 'Ethnicity_Hispanic')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-hispanic/lcs-dp-hispanic-' + str(i), '../experiments/lcs-original', 'Ethnicity_Hispanic')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-hispanic', '../experiments/lcs-original', 'Ethnicity_Hispanic')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-age90/lcs-dp-age90-' + str(i), 'lung', 'Age>=90')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-age90/lcs-dp-age90-' + str(i), '../experiments/lcs-original', 'Age>=90')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-age90', '../experiments/lcs-original', 'Age>=90')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-3040/lcs-dp-3040-' + str(i), 'lung', '30<=Age<40')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-3040/lcs-dp-3040-' + str(i), '../experiments/lcs-original', '30<=Age<40')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-3040', '../experiments/lcs-original', '30<=Age<40')

In [ ]:
for i in range(1, 6):
    eval_all_units_whole_group('../experiments/lcs-dp-8090/lcs-dp-8090-' + str(i), 'lung', '80<=Age<90')
    select_dp_unit_whole_group_threshold('../experiments/lcs-dp-8090/lcs-dp-8090-' + str(i), '../experiments/lcs-original', '80<=Age<90')
average_5_runs_dp_whole_group_threshold('../experiments/lcs-dp-8090', '../experiments/lcs-original', '80<=Age<90')

In [ ]:
eval_all_units_whole_group('../experiments/bcs-dpreweight-asian/bcs-dpreweight-asian-1', 'breast', 'Ethnicity_Asian')
select_dp_unit_whole_group_threshold('../experiments/bcs-dpreweight-asian/bcs-dpreweight-asian-1', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
eval_all_units_whole_group('../experiments/bcs-dpreweight-4050/bcs-dpreweight-4050-1', 'breast', '40<=Age<50')
select_dp_unit_whole_group_threshold('../experiments/bcs-dpreweight-4050/bcs-dpreweight-4050-1', '../experiments/bcs-original', '40<=Age<50')

In [ ]:
for i in range(2,6):
    eval_all_units_whole_group('../experiments/bcs-dpreweight-asian/bcs-dpreweight-asian-' + str(i), 'breast', 'Ethnicity_Asian')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dpreweight-asian/bcs-dpreweight-asian-' + str(i), '../experiments/bcs-original', 'Ethnicity_Asian')

    eval_all_units_whole_group('../experiments/bcs-dpreweight-4050/bcs-dpreweight-4050-' + str(i), 'breast', '40<=Age<50')
    select_dp_unit_whole_group_threshold('../experiments/bcs-dpreweight-4050/bcs-dpreweight-4050-' + str(i), '../experiments/bcs-original', '40<=Age<50')

In [ ]:
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dpreweight-asian', '../experiments/bcs-original', 'Ethnicity_Asian')
average_5_runs_dp_whole_group_threshold('../experiments/bcs-dpreweight-4050', '../experiments/bcs-original', '40<=Age<50')

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
tf.test.gpu_device_name()

In [ ]:
import shap
shap.__version__

In [ ]:
################################
# old subgroup threshold setting

In [ ]:
############################
## debug and run
############################
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-2', 'breast', 'Ethnicity_Black')

In [ ]:
units = ['9u', '10u', '12u', '14u', '16u', '18u', '20u']
folder_name = '../experiments/bcs-dp-black/bcs-dp-black-2'
cancer = 'breast'
group_name = 'Ethnicity_Black'
    
for u in units:
    folder = folder_name + '/' + folder_name.split('/')[-1][:-2] + '-' + u

    calibrate_results(folder, 25)
    generate_everything_all_epochs(folder, 25, cancer)
    select_model(folder, 25, group_name)

In [ ]:
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-3', 'breast', 'Ethnicity_Black')

In [ ]:
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-4', 'breast', 'Ethnicity_Black')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-black/bcs-dp-black-2', 'Ethnicity_Black')

In [ ]:
eval_all_units('../experiments/bcs-dp-asian/bcs-dp-asian-2', 'breast', 'Ethnicity_Asian')

In [ ]:
eval_all_units('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-2', 'breast', 'Ethnicity_Hispanic')

In [ ]:
eval_all_units('../experiments/bcs-dp-age30/bcs-dp-age30-1', 'breast', 'Age<30')

In [ ]:
eval_all_units('../experiments/bcs-dp-age90/bcs-dp-age90-1', 'breast', 'Age>=90')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-asian/bcs-dp-asian-2', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-2', '../experiments/bcs-original', 'Ethnicity_Hispanic')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-age30/bcs-dp-age30-1', '../experiments/bcs-original', 'Age<30')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-age90/bcs-dp-age90-1', '../experiments/bcs-original', 'Age>=90')

In [ ]:
eval_all_units('../experiments/bcs-dp-asian/bcs-dp-asian-3', 'breast', 'Ethnicity_Asian')
eval_all_units('../experiments/bcs-dp-asian/bcs-dp-asian-4', 'breast', 'Ethnicity_Asian')
eval_all_units('../experiments/bcs-dp-asian/bcs-dp-asian-5', 'breast', 'Ethnicity_Asian')

In [ ]:
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-3', 'breast', 'Ethnicity_Black')
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-4', 'breast', 'Ethnicity_Black')
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-5', 'breast', 'Ethnicity_Black')

In [ ]:
eval_all_units('../experiments/bcs-dp-age90/bcs-dp-age90-2', 'breast', 'Age>=90')
eval_all_units('../experiments/bcs-dp-age90/bcs-dp-age90-3', 'breast', 'Age>=90')
eval_all_units('../experiments/bcs-dp-age90/bcs-dp-age90-4', 'breast', 'Age>=90')
eval_all_units('../experiments/bcs-dp-age90/bcs-dp-age90-5', 'breast', 'Age>=90')

In [ ]:
eval_all_units('../experiments/bcs-dp-age30/bcs-dp-age30-2', 'breast', 'Age<30')
eval_all_units('../experiments/bcs-dp-age30/bcs-dp-age30-3', 'breast', 'Age<30')
eval_all_units('../experiments/bcs-dp-age30/bcs-dp-age30-4', 'breast', 'Age<30')
eval_all_units('../experiments/bcs-dp-age30/bcs-dp-age30-5', 'breast', 'Age<30')

In [ ]:
eval_all_units('../experiments/bcs-dp-asian/bcs-dp-asian-1', 'breast', 'Ethnicity_Asian')
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-1', 'breast', 'Ethnicity_Black')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-black/bcs-dp-black-1', '../experiments/bcs-original', 'Ethnicity_Black')

In [ ]:
select_dp_unit_subgroup_threshold('../experiments/bcs-dp-asian/bcs-dp-asian-1', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
for i in range(2, 6):
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-black/bcs-dp-black-' + str(i), '../experiments/bcs-original', 'Ethnicity_Black')
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-asian/bcs-dp-asian-' + str(i), '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
eval_all_units('../experiments/bcs-dp-black/bcs-dp-black-5', 'breast', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    eval_all_units('../experiments/lcs-dp-asian/lcs-dp-asian-' + str(i), 'lung', 'Ethnicity_Asian')
    eval_all_units('../experiments/lcs-dp-black/lcs-dp-black-' + str(i), 'lung', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    eval_all_units('../experiments/bcs-dp-3040/bcs-dp-3040-' + str(i), 'breast', '30<=Age<40')
    eval_all_units('../experiments/bcs-dp-4050/bcs-dp-4050-' + str(i), 'breast', '40<=Age<50')

In [ ]:
for i in range(1, 6):
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-age30/bcs-dp-age30-' + str(i), '../experiments/bcs-original', 'Age<30')
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-age90/bcs-dp-age90-' + str(i), '../experiments/bcs-original', 'Age>=90')

In [ ]:
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-asian', 'Ethnicity_Asian')
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-black', 'Ethnicity_Black')

In [ ]:
for i in range(1, 6):
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-' + str(i), '../experiments/bcs-original', 'Ethnicity_Hispanic')

In [ ]:
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-hispanic', '../experiments/bcs-original', 'Ethnicity_Hispanic')

In [ ]:
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-age30', '../experiments/bcs-original', 'Age<30')
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-age90', '../experiments/bcs-original', 'Age>=90')

In [ ]:
for i in range(1, 6):
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-black/lcs-dp-black-' + str(i), '../experiments/lcs-original', 'Ethnicity_Black')
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-asian/lcs-dp-asian-' + str(i), '../experiments/lcs-original', 'Ethnicity_Asian')

In [ ]:
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-asian', '../experiments/lcs-original', 'Ethnicity_Asian')
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-black', '../experiments/lcs-original', 'Ethnicity_Black')

In [ ]:
eval_all_units('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-1', 'breast', 'Ethnicity_Hispanic')

In [ ]:
eval_all_units('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-3', 'breast', 'Ethnicity_Hispanic')
eval_all_units('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-4', 'breast', 'Ethnicity_Hispanic')
eval_all_units('../experiments/bcs-dp-hispanic/bcs-dp-hispanic-5', 'breast', 'Ethnicity_Hispanic')

In [ ]:
for i in range(1, 6):
    eval_all_units('../experiments/lcs-dp-hispanic/lcs-dp-hispanic-' + str(i), 'lung', 'Ethnicity_Hispanic')
    eval_all_units('../experiments/lcs-dp-age90/lcs-dp-age90-' + str(i), 'lung', 'Age>=90')

In [ ]:
for i in range(1, 6):
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-hispanic/lcs-dp-hispanic-' + str(i), '../experiments/lcs-original', 'Ethnicity_Hispanic')
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-age90/lcs-dp-age90-' + str(i), '../experiments/lcs-original', 'Age>=90')

In [ ]:
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-hispanic', '../experiments/lcs-original', 'Ethnicity_Hispanic')
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-age90', '../experiments/lcs-original', 'Age>=90')

In [ ]:
for i in range(1, 6):
    eval_all_units('../experiments/lcs-dp-8090/lcs-dp-8090-' + str(i), 'lung', '80<=Age<90')
    eval_all_units('../experiments/lcs-dp-3040/lcs-dp-3040-' + str(i), 'lung', '30<=Age<40')

In [ ]:
for i in range(1, 6):  
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-8090/lcs-dp-8090-' + str(i), '../experiments/lcs-original', '80<=Age<90')
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-8090', '../experiments/lcs-original', '80<=Age<90')

In [ ]:
for i in range(1, 6):  
    select_dp_unit_subgroup_threshold('../experiments/lcs-dp-3040/lcs-dp-3040-' + str(i), '../experiments/lcs-original', '30<=Age<40')
average_5_runs_dp_subgroup_threshold('../experiments/lcs-dp-3040', '../experiments/lcs-original', '30<=Age<40')

In [ ]:
for i in range(1, 6):  
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-3040/bcs-dp-3040-' + str(i), '../experiments/bcs-original', '30<=Age<40')
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-3040', '../experiments/bcs-original', '30<=Age<40')

In [ ]:
for i in range(1, 6):  
    select_dp_unit_subgroup_threshold('../experiments/bcs-dp-4050/bcs-dp-4050-' + str(i), '../experiments/bcs-original', '40<=Age<50')
average_5_runs_dp_subgroup_threshold('../experiments/bcs-dp-4050', '../experiments/bcs-original', '40<=Age<50')

In [ ]:

((0.01212) ** 2 / 5 + (0.01947) ** 2 / 5) ** (1/2)

In [ ]:
((0.01212) ** 2 + (0.01947) ** 2) ** (1/2)